*Expriments For In-domain Scenarious of BERT and DistilBERT (split 80/10/10)*

In [1]:
#Due to the computing power available from the Google Collab, we have to mount to the drive to save the model files.
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
#Here we control the randomness of our experiments, therefore we can be reproducable for further research.
import random
import numpy as np
import torch

def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(42)

In [3]:
#Addding to the reprodicibility and transparency of the model training, we can utilize the WandB library to check on important metrics of the experiments.
#Use of WandB is not neccessary, but it will help us to make good decisions and not waste valuable resources.
import wandb
wandb.login()

import os
os.environ["WANDB_PROJECT"] = "moral-foundations-all-NEW-5epoch"



wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: battemuulenn (battemuulenn-university-of-amsterdam) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [4]:
import pandas as pd
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split

df1 = pd.read_csv('filtered_moral_annotations_MFRC.csv')
df2 = pd.read_csv('filtered_moral_annotations_MFTC.csv')

shared_labels = ["care", "fairness", "loyalty", "authority", "non-moral"]

def filter_and_normalize_labels(label_str, shared_labels):
    labels = [l.strip().lower() for l in str(label_str).split(',')]
    return [l for l in labels if l in shared_labels]

df1['labels'] = df1['annotation'].apply(lambda x: filter_and_normalize_labels(x, shared_labels))
df2['labels'] = df2['annotation'].apply(lambda x: filter_and_normalize_labels(x, shared_labels))
df1 = df1[df1['labels'].map(len) > 0].reset_index(drop=True)
df2 = df2[df2['labels'].map(len) > 0].reset_index(drop=True)

mlb = MultiLabelBinarizer(classes=shared_labels)
mlb.fit([shared_labels])
df1['label_vec'] = mlb.transform(df1['labels']).tolist()
df2['label_vec'] = mlb.transform(df2['labels']).tolist()

# 80/10/10 split
train_val_df1, test_df1 = train_test_split(df1, test_size=0.1, random_state=42)
train_df1, val_df1 = train_test_split(train_val_df1, test_size=1/9, random_state=42)
train_val_df2, test_df2 = train_test_split(df2, test_size=0.1, random_state=42)
train_df2, val_df2 = train_test_split(train_val_df2, test_size=1/9, random_state=42)


In [5]:
from torch.utils.data import Dataset
from transformers import (
    DistilBertTokenizerFast, DistilBertForSequenceClassification,
    BertTokenizerFast, BertForSequenceClassification,
    Trainer, TrainingArguments
)
from sklearn.metrics import f1_score, accuracy_score
import torch
import numpy as np

class TextDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length=128):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        encoding = self.tokenizer(
            self.texts[idx],
            truncation=True,
            padding='max_length',
            max_length=self.max_length,
            return_tensors='pt'
        )
        item = {key: val.squeeze(0) for key, val in encoding.items()}
        item['labels'] = torch.tensor(self.labels[idx], dtype=torch.float)
        return item

def get_model_and_tokenizer(model_name, num_labels):
    if 'distilbert' in model_name:
        tokenizer = DistilBertTokenizerFast.from_pretrained(model_name)
        model = DistilBertForSequenceClassification.from_pretrained(
            model_name, num_labels=num_labels, problem_type="multi_label_classification"
        )
    else:
        tokenizer = BertTokenizerFast.from_pretrained(model_name)
        model = BertForSequenceClassification.from_pretrained(
            model_name, num_labels=num_labels, problem_type="multi_label_classification"
        )
    return tokenizer, model

def get_datasets(df, tokenizer):
    return TextDataset(
        texts=df['text'].tolist(),
        labels=df['label_vec'].tolist(),
        tokenizer=tokenizer
    )

def compute_metrics(pred):
    logits, labels = pred
    probs = 1 / (1 + np.exp(-logits))
    preds = (probs > 0.5).astype(int)
    f1 = f1_score(labels, preds, average='micro', zero_division=0)
    acc = accuracy_score(labels, preds)
    return {'f1': f1, 'accuracy': acc}

def train_and_evaluate_scenario(
    train_df, val_df, model_name, mlb, epochs=5,
    run_name=None, save_model=False, save_dir=None, scenario_name=None
):
    num_labels = len(mlb.classes_)
    tokenizer, model = get_model_and_tokenizer(model_name, num_labels)
    train_dataset = get_datasets(train_df, tokenizer)
    val_dataset = get_datasets(val_df, tokenizer)

    training_args = TrainingArguments(
        output_dir='./results',
        num_train_epochs=epochs,
        per_device_train_batch_size=8,
        per_device_eval_batch_size=16,
        logging_steps=50,
        learning_rate=2e-5,
        weight_decay=0.01,
        disable_tqdm=False,
        report_to="none",
        run_name=run_name,
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset,
        compute_metrics=lambda p: compute_metrics((p.predictions, p.label_ids)),
    )

    trainer.train()
    eval_results = trainer.evaluate()
    print(f"Validation results for {model_name}: {eval_results}")

    # Save model and tokenizer if requested
    if save_model and save_dir is not None:
        save_path = f"/content/drive/MyDrive/{save_dir}"
        import os
        os.makedirs(save_path, exist_ok=True)
        model.save_pretrained(save_path)
        tokenizer.save_pretrained(save_path)
        print(f"Model and tokenizer saved to {save_path}")

    eval_results['scenario'] = scenario_name if scenario_name else run_name

    return eval_results, trainer, tokenizer


In [6]:
metrics_path = "/content/drive/MyDrive/model_performance_metrics_NEW_5epoch_CORRECT.csv"

eval_results_distilbert_mfrc_mfrc, trainer_distilbert_mfrc_mfrc, tokenizer_distilbert_mfrc_mfrc = train_and_evaluate_scenario(
    train_df1, val_df1, 'distilbert-base-uncased', mlb,
    epochs=5,
    run_name="distilbert-mfrc-mfrc",
    scenario_name="DistilBERT MFRC→MFRC (in-domain split, validation)"
)

test_dataset = get_datasets(test_df1, tokenizer_distilbert_mfrc_mfrc)
final_test_results = trainer_distilbert_mfrc_mfrc.evaluate(test_dataset)
print("Final test results (DistilBERT MFRC→MFRC):", final_test_results)

preds = trainer_distilbert_mfrc_mfrc.predict(test_dataset)
pred_logits = preds.predictions
pred_probs = 1 / (1 + np.exp(-pred_logits))
pred_bin = (pred_probs > 0.5).astype(int)
pred_labels = mlb.inverse_transform(pred_bin)
test_results_df = test_df1.copy()
test_results_df['pred_labels'] = pred_labels
test_results_df['pred_label_vec'] = list(pred_bin)
test_results_df.to_pickle("/content/drive/MyDrive/test_results_distilbert_mfrc_mfrc_split.pkl")
print("Test results DataFrame saved to /content/drive/MyDrive/test_results_distilbert_mfrc_mfrc_split.pkl")

final_test_results['scenario'] = "DistilBERT MFRC→MFRC (in-domain split, test)"
metrics_df = pd.DataFrame([final_test_results])
import os
if not os.path.exists(metrics_path):
    metrics_df.to_csv(metrics_path, index=False)
else:
    metrics_df.to_csv(metrics_path, mode='a', header=False, index=False)
print(f"Metrics saved to {metrics_path}")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Step,Training Loss
50,0.506800
100,0.407600
150,0.372000
200,0.352200
250,0.369500
300,0.368300
350,0.346300
400,0.387000
450,0.343700
500,0.353500


Validation results for distilbert-base-uncased: {'eval_loss': 0.368299275636673, 'eval_f1': 0.6915294584407855, 'eval_accuracy': 0.6501877346683355, 'eval_runtime': 5.9107, 'eval_samples_per_second': 811.071, 'eval_steps_per_second': 50.755, 'epoch': 5.0}
Final test results (DistilBERT MFRC→MFRC): {'eval_loss': 0.3718898296356201, 'eval_f1': 0.6866144852868324, 'eval_accuracy': 0.6451814768460575, 'eval_runtime': 6.2252, 'eval_samples_per_second': 770.09, 'eval_steps_per_second': 48.191, 'epoch': 5.0}
Test results DataFrame saved to /content/drive/MyDrive/test_results_distilbert_mfrc_mfrc_split.pkl
Metrics saved to /content/drive/MyDrive/model_performance_metrics_NEW_5epoch_CORRECT.csv


In [7]:
eval_results_distilbert_mftc_mftc, trainer_distilbert_mftc_mftc, tokenizer_distilbert_mftc_mftc = train_and_evaluate_scenario(
    train_df2, val_df2, 'distilbert-base-uncased', mlb,
    epochs=5,
    run_name="distilbert-mftc-mftc",
    scenario_name="DistilBERT MFTC→MFTC (in-domain split, validation)"
)

test_dataset = get_datasets(test_df2, tokenizer_distilbert_mftc_mftc)
final_test_results = trainer_distilbert_mftc_mftc.evaluate(test_dataset)
print("Final test results (DistilBERT MFTC→MFTC):", final_test_results)

preds = trainer_distilbert_mftc_mftc.predict(test_dataset)
pred_logits = preds.predictions
pred_probs = 1 / (1 + np.exp(-pred_logits))
pred_bin = (pred_probs > 0.5).astype(int)
pred_labels = mlb.inverse_transform(pred_bin)
test_results_df = test_df2.copy()
test_results_df['pred_labels'] = pred_labels
test_results_df['pred_label_vec'] = list(pred_bin)
test_results_df.to_pickle("/content/drive/MyDrive/test_results_distilbert_mftc_mftc_split.pkl")
print("Test results DataFrame saved to /content/drive/MyDrive/test_results_distilbert_mftc_mftc_split.pkl")

final_test_results['scenario'] = "DistilBERT MFTC→MFTC (in-domain split, test)"
metrics_df = pd.DataFrame([final_test_results])
if not os.path.exists(metrics_path):
    metrics_df.to_csv(metrics_path, index=False)
else:
    metrics_df.to_csv(metrics_path, mode='a', header=False, index=False)
print(f"Metrics saved to {metrics_path}")


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Step,Training Loss
50,0.491100
100,0.398300
150,0.329800
200,0.308600
250,0.291700
300,0.317500
350,0.301100
400,0.281400
450,0.290800
500,0.249200


Validation results for distilbert-base-uncased: {'eval_loss': 0.283378005027771, 'eval_f1': 0.772093572778828, 'eval_accuracy': 0.7450597706757746, 'eval_runtime': 9.6577, 'eval_samples_per_second': 848.858, 'eval_steps_per_second': 53.118, 'epoch': 5.0}
Final test results (DistilBERT MFTC→MFTC): {'eval_loss': 0.28576213121414185, 'eval_f1': 0.77209797657082, 'eval_accuracy': 0.7418882654305928, 'eval_runtime': 10.2401, 'eval_samples_per_second': 800.575, 'eval_steps_per_second': 50.097, 'epoch': 5.0}
Test results DataFrame saved to /content/drive/MyDrive/test_results_distilbert_mftc_mftc_split.pkl
Metrics saved to /content/drive/MyDrive/model_performance_metrics_NEW_5epoch_CORRECT.csv


In [8]:
eval_results_bert_mfrc_mfrc, trainer_bert_mfrc_mfrc, tokenizer_bert_mfrc_mfrc = train_and_evaluate_scenario(
    train_df1, val_df1, 'bert-base-uncased', mlb,
    epochs=5,
    run_name="bert-mfrc-mfrc",
    scenario_name="BERT MFRC→MFRC (in-domain split, validation)"
)

test_dataset = get_datasets(test_df1, tokenizer_bert_mfrc_mfrc)
final_test_results = trainer_bert_mfrc_mfrc.evaluate(test_dataset)
print("Final test results (BERT MFRC→MFRC):", final_test_results)

preds = trainer_bert_mfrc_mfrc.predict(test_dataset)
pred_logits = preds.predictions
pred_probs = 1 / (1 + np.exp(-pred_logits))
pred_bin = (pred_probs > 0.5).astype(int)
pred_labels = mlb.inverse_transform(pred_bin)
test_results_df = test_df1.copy()
test_results_df['pred_labels'] = pred_labels
test_results_df['pred_label_vec'] = list(pred_bin)
test_results_df.to_pickle("/content/drive/MyDrive/test_results_bert_mfrc_mfrc_split.pkl")
print("Test results DataFrame saved to /content/drive/MyDrive/test_results_bert_mfrc_mfrc_split.pkl")

final_test_results['scenario'] = "BERT MFRC→MFRC (in-domain split, test)"
metrics_df = pd.DataFrame([final_test_results])
if not os.path.exists(metrics_path):
    metrics_df.to_csv(metrics_path, index=False)
else:
    metrics_df.to_csv(metrics_path, mode='a', header=False, index=False)
print(f"Metrics saved to {metrics_path}")


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Step,Training Loss
50,0.504100
100,0.405500
150,0.368800
200,0.348400
250,0.366200
300,0.355800
350,0.338100
400,0.382600
450,0.337500
500,0.357000


Validation results for bert-base-uncased: {'eval_loss': 0.3862771689891815, 'eval_f1': 0.6834052569635151, 'eval_accuracy': 0.6366291197329996, 'eval_runtime': 10.117, 'eval_samples_per_second': 473.856, 'eval_steps_per_second': 29.653, 'epoch': 5.0}
Final test results (BERT MFRC→MFRC): {'eval_loss': 0.3794969916343689, 'eval_f1': 0.6850493790945537, 'eval_accuracy': 0.6403838130997079, 'eval_runtime': 10.3424, 'eval_samples_per_second': 463.53, 'eval_steps_per_second': 29.007, 'epoch': 5.0}
Test results DataFrame saved to /content/drive/MyDrive/test_results_bert_mfrc_mfrc_split.pkl
Metrics saved to /content/drive/MyDrive/model_performance_metrics_NEW_5epoch_CORRECT.csv


In [9]:
eval_results_bert_mftc_mftc, trainer_bert_mftc_mftc, tokenizer_bert_mftc_mftc = train_and_evaluate_scenario(
    train_df2, val_df2, 'bert-base-uncased', mlb,
    epochs=5,
    run_name="bert-mftc-mftc",
    scenario_name="BERT MFTC→MFTC (in-domain split, validation)"
)

test_dataset = get_datasets(test_df2, tokenizer_bert_mftc_mftc)
final_test_results = trainer_bert_mftc_mftc.evaluate(test_dataset)
print("Final test results (BERT MFTC→MFTC):", final_test_results)

preds = trainer_bert_mftc_mftc.predict(test_dataset)
pred_logits = preds.predictions
pred_probs = 1 / (1 + np.exp(-pred_logits))
pred_bin = (pred_probs > 0.5).astype(int)
pred_labels = mlb.inverse_transform(pred_bin)
test_results_df = test_df2.copy()
test_results_df['pred_labels'] = pred_labels
test_results_df['pred_label_vec'] = list(pred_bin)
test_results_df.to_pickle("/content/drive/MyDrive/test_results_bert_mftc_mftc_split.pkl")
print("Test results DataFrame saved to /content/drive/MyDrive/test_results_bert_mftc_mftc_split.pkl")

final_test_results['scenario'] = "BERT MFTC→MFTC (in-domain split, test)"
metrics_df = pd.DataFrame([final_test_results])
if not os.path.exists(metrics_path):
    metrics_df.to_csv(metrics_path, index=False)
else:
    metrics_df.to_csv(metrics_path, mode='a', header=False, index=False)
print(f"Metrics saved to {metrics_path}")


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Step,Training Loss
50,0.496500
100,0.397000
150,0.346800
200,0.315400
250,0.295300
300,0.312200
350,0.313000
400,0.297800
450,0.293000
500,0.258100


Validation results for bert-base-uncased: {'eval_loss': 0.2915445864200592, 'eval_f1': 0.7686242203130517, 'eval_accuracy': 0.7404244937789705, 'eval_runtime': 16.6046, 'eval_samples_per_second': 493.719, 'eval_steps_per_second': 30.895, 'epoch': 5.0}
Final test results (BERT MFTC→MFTC): {'eval_loss': 0.2910127341747284, 'eval_f1': 0.7677669446728533, 'eval_accuracy': 0.7374969504757258, 'eval_runtime': 17.1312, 'eval_samples_per_second': 478.542, 'eval_steps_per_second': 29.945, 'epoch': 5.0}
Test results DataFrame saved to /content/drive/MyDrive/test_results_bert_mftc_mftc_split.pkl
Metrics saved to /content/drive/MyDrive/model_performance_metrics_NEW_5epoch_CORRECT.csv
